# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This problem is classification. There is a binary output value for our learning algorithm - pass or fail. If there was a subtle change to the problem, for instance calculate the expected grade of the student, then we would have a regression problem since the grade of a student is a continuous variable. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = 100 * float(n_passed) / float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=7)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

**Decision Trees**

**industry application: ** It took a bit of binging to find some real-world applications but eventually we banged out some results. Wikipedia provided a lot of information on the mechanics of decision trees but I ended up looking at scholarly works for uses of Decision trees in real life [here](http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html). The medical references interested me the most and I followed it to this [pdf](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.109.7119&rep=rep1&type=pdf). The article discusses how decision trees and naive bayes classifiers are used to determine whether a patient has a type of serious illness. The main conclusion regarding decision trees was that though it outperformed physicians in determining the patient's illness, it wasn't very reliable since it only took the most important attributes into account and didn't seem to be very holistic. (Bottom of page 8). 

**strengths: ** From the essay, I realized (or was rereminded) that decision trees can deal with continuous values while Bayesian classifiers have trouble with continuous variables. Another advantage is that since decision trees can deal with numerical and categorical variables well, we don't need to do to preprocessing that created dummy variables. (But since we have, it seems like Bayes would be a good learner for this dataset). 

**weaknesses: ** Decision trees can become weak when dealing with things like XOR and other odd booleans (as was shown in some of the instructional videos). The tree can become pretty big when it tries to learn those underlying relationships. Decision trees also struggle with providing reliability with thier decisions. As was shown in the paper, decision trees don't provide anything like confidence intervals. This property also makes them somewhat non-robust, where they may change drastically with slight changes in training data.

** good candidate for problem: ** Since the dataset seems to be relatively large (lots of features) and there are some numerical features that I feel would strongly influence whether a student passes (studytime, freetime) this learner is an appropriate fit for the data. 

**Adaboost**

**industry application: ** From the course lectures it would seem that spam detection seems like a good use of ensemble methods. When a problem is too hard or complex to understand with just one classifier, it makes sense to use many classifiers. If ensemble methods are used in spam, then it's likely that they're used to learn languages themselves. I had a friend who was working with Natural Language Processing, and that problem came to mind. A quick search shows that people have taken in interest in using ensemble methods to try and learn the many rules of common languages and that ensemble methods show improvement over things like decision trees. A paper on this topic is [here](https://www.cs.cornell.edu/home/cardie/papers/hlt-emnlp05-art.pdf). We're going to use boosting as our algorithm for combining the learners so that we're training on sections of the data we've had trouble with. Adaboost is a nice goto algorithm because it combines boosting and decision trees. 

**Strengths: ** From the paper a strength seems to be that Adaboost performs well in situations when there are many reasons for determining the output of a classifier. Adaboost can also work well on large datasets because they simply break down the dataset into an arbitrary number of training sets for each of the decistion trees built into the ensemble. 

**Weaknesses: ** Adaboost can sometimes overfit. A decision tree may overfit by assigning one datapoint to a leaf node, and as such overfitting the training set. Since adaboost is built with decision trees, the error is likely to carry over. When a training set is broken down, certain attributes might be randomly emphasized and as such the decision trees pick up rules that shouldn't really exist. Our dataset has many features, but relatively few data points. We won't be able to split it into too many training sets. 

** good candidate for problem: ** Since it seems likely that with 30 or 40 features there could be several combinations of complex rules that could determine if a student passes or fails, I expect Adaboost to be able to be as good as or better than other classifiers. 

** Gaussian Naive Bayes **

** industry application: ** Naive Bayes seems to be pretty widespread in industry use. The reason for this is that it scales linearly with the number of features that are used to classify the output variable. For this reason, it can be used as a method that has some reliability in situations with many features, such as medical diagnosis. (Referenced from wikipedia and the paper linked when discussing decision trees). 

**strengths** Bayes provides some kind of reliability and stability when determining an output. Since each feature is weighed independently against the output, we can take into account how important each feature is in determining what the output should be, and how reliable that feature is when predicting the output. Since the learner scales linearly with the number of features, it manages large datasets relatively quickly. 

**weaknesses** Bayes makes a strong assumption: the input features are all independent. This could break the classification and hence this is why it's a "naive" classifier. Another issue is that the classifier tends to "bin" continuous variables into binary categories. This could throw out much information that would be used in the classificiation. 

**good candidate for problem: ** The main arguing point is that bayes is robust and works in linear time to learn the dataset. This is a huge selling point, even if it means lower accuracy. Since the data has already been preprocessed to break up the categorical variables into dummy variables, the data is poised for use with a bayes classifier. 

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [10]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [13]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state=7)
clf_C = AdaBoostClassifier(random_state=7)

# TODO: Set up the training set sizes
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train
y_train_300 = y_train

classifiers = [clf_A, clf_B, clf_C]

for learner in classifiers:
    train_predict(learner, X_train_100, y_train_100, X_test, y_test)
    train_predict(learner, X_train_200, y_train_200, X_test, y_test)
    train_predict(learner, X_train_300, y_train_300, X_test, y_test)
    
# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.3077.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.2791.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.7985.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8299.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.7873.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.8258.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6457.
Training a DecisionTreeClassifier using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gaussian Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0020                 |    0                    |     0.3077             |       0.2791          |
| 200               |        0.0020          |          0.0010              |        0.7985          |          0.8299       |
| 300               |         0.0020                |         0.0010               |        0.7873          |    0.8258      |

** Classifer 2 - Decision Tree Classifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0020                 |       0                 |      1            |         0.6457        |
| 200               |     0.0030             |         0.0010               |        1          |         0.8116        |
| 300               |        0.0040                 |        0                |         1         |     0.7808     |

** Classifer 3 - AdaBoost **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.1430             |        0.0050                |         0.9848         |       0.7639          |
| 200               |          0.1530               |       0.0050                 |         0.8647         |       0.7778  |
| 300               |           0.1600       |      0.0070                  |        0.8544          |      0.7808           |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** 
I chose Naive Bayes as the best model. I chose it because the Decision Tree overfit the training set and because the Adaboost took more time and didn't perform as well as Naive Bayes. There are more factors to consider, but accuracy and runtime are the most important because we want our model to classify correctly and a smaller runtime means a cheaper implementation. 

The Decision Tree and Ensemble methods have the issue of being more taxing on resources. A decision tree can use up memory quickly when creating complex relations within the tree and Ensemble methods also use up lots of resources because they are creating many different classifiers, each with their own objects. Hence, the cheapest and best performing classifier seems to be Bayes. Naive Bayes does well in terms of performance because it learns in linear time and also predicts in linear time as well.

In terms of the f1 score, we're most concerned with minimizing the differences between training and testing scores. The Bayes classifier is only off by a maximum of 5 percent, while decision trees and Ensemble methods are off by more than 20 and 7 percent respectively. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** The main goal of the learner is to solve the problem of estimating whether a student will fail or not. Using the given data the model determines probabilities associated with each feature and whether the student passes or fails. Features are the data we collect about each student, whether they have a job, internet, are in a relationship, etc. The model precomputes probabilities of the form: "The student has internet, given that he or she passed" and "The student does not have internet, given that he or she passed." and also "The student has internet, given that he or she failed" and "The student does not have internet, given he or she failed". This is the process of training the learner. Once all of these probabilities are computed for each feature, we can make a prediction. If we want to know whether a student is expected to pass or fail with a given set of features, we can compute the probability that he or she will pass and the probability that he or she will fail using the precomputed probabilities for each feature. Once that is done, we take whichever number is higher, and then classify the student as pass or fail. 

The model trains quickly because it only ever has to think about each feature once in relation to whether the student passes or fails. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [58]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'max_depth' : [ 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]}

# TODO: Initialize the classifier
clf = DecisionTreeClassifier(random_state=7)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y=y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8460.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.8289.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** I know this looks bad, tuning the DecisionTreeClassifier and not the GaussianNB but the issue was that I couldn't see any way where GaussianNB would be helped with parameter tuning since the model is so simple and only has two parameters - both of which are calculated from the data. The DecisionTreeClassifier has many more interesting parameters and it seemed like a good idea to try some manual pruning to see which tree depth gave the best f1 scores. The reason it made sense to prune the DecisionTree is that during the training phase, it would overfit and match the training data perfectly but then suffer consequently on the testing data. Some pruning helped the f1 score on the testing set which what we're looking for and brought the training f1 score close to the testing f1 score. This consistency shows that we're neither overfitting or underfitting. The untuned tree overfit, while the tuned tree doesn't. We see an f1 score gain of about 5 percent. (roughly 83 - 78). 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.